<a href="https://colab.research.google.com/github/yasirabd/research-diagnostic-turbine/blob/main/Diagnostics_Steam_Turbine_with_VBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Author: Yasir Abdur Rohman*<br>
**Property of PT Indonesia Power & Lab Getaran & Diagnosis Mesin Undip**

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial import distance

from sklearn.metrics import mean_squared_error

import plotly.express as px
import plotly.graph_objs as go

In [ ]:
# load dataset

# sla 18
df = pd.read_excel("/content/drive/MyDrive/Turbine Bearing/Data/Data_SLA_Unit_1_2018.xlsx")

In [ ]:
# columns in SLA 2018
list(df)

['Tanggal',
 'Generator Output (MW)',
 'Generator VAR (MVAR)',
 'Turb Bearing Vib 1 (uM)',
 'Turb Bearing Vib 2 (uM)',
 'Turb Bearing Vib 3 (uM)',
 'Turb Bearing Vib 4 (uM)',
 'Turb Bearing Vib 5 (uM)',
 'Turb Bearing Vib 6 (uM)',
 'Gen Bearing Vib 7 (uM)',
 'Gen Bearing Vib 8 (uM)',
 'Gen Bearing Vib 9 (uM)',
 'Lube Oil Inlet Cooler Temp Ct (Deg)',
 'Lube Oil Outlet Cooler Temp Ct (Deg)',
 'No 1 Bearing Oil Drain Temp (Deg)',
 'No 2 Bearing Oil Drain Temp (Deg)',
 'No 3 Bearing Oil Drain Temp (Deg)',
 'No 4 Bearing Oil Drain Temp (Deg)',
 'No 5 Bearing Oil Drain Temp (Deg)',
 'No 6 Bearing Oil Drain Temp (Deg)',
 'No 7 Bearing Oil Drain Temp (Deg)',
 'No 8 Bearing Oil Drain Temp (Deg)',
 'No 9 Bearing Oil Drain Temp (Deg)',
 'Thrust Bearing Oil Drain Temp FR (Deg)',
 'Thrust Bearing Oil Drain Temp RR (Deg)',
 'No 1 Turb Bearing Metal Temp A ( Deg )',
 'No 1 Turb Bearing Metal Temp B ( Deg )',
 'No 2 Turb Bearing Metal Temp A ( Deg )',
 'No 2 Turb Bearing Metal Temp B ( Deg )',
 'No 3 

# ST Journal Bearing 1 - Cooling Loss

Description:
- ST Journal Bearing #1 - Cooling Loss.  This fault indicates an abnormal change in the cooling of the supply oil to the bearings.  Primary indicators are: Journal Bearing Oil Outlet Temperature (Oil Return) is high; and Bearing Metal Temperature is high.   Priority 4 (lowest) is triggered when either Lube Oil Outlet Temperature is high or Bearing Metal Temperature is high.  The priority escalates depending on the severity of the change, and the number of sensors.  This fault triggers  Priority 2, 3 & 4.

Input Tags:
- LUBE_OIL_COOLER_OIL_OUTLET_TEMP
- GROSS_LOAD
- LUBE_OIL_COOLER_OIL_OUTLET_PRESS
- BRG_1_MTL_TEMP_1 
- AMBIENT_TEMP
- BRG_1_MTL_TEMP_2 
- BRG_1_MTL_TEMP_4 
- BRG_1_MTL_TEMP_3
- BRG_1_OIL_DRAIN_TEMP

In [ ]:
selected_cols = ['Generator Output (MW)', # gross load
                 'Lube Oil Outlet Cooler Temp Ct (Deg)',
                 'No 1 Bearing Oil Drain Temp (Deg)', 
                 'No 1 Turb Bearing Metal Temp A ( Deg )',
                 'No 1 Turb Bearing Metal Temp B ( Deg )',]

### VBM

In [ ]:
def scipy_distance(vector1, vector2, dist='euclidean'):
    if dist == 'euclidean':
        return distance.euclidean(vector1, vector2)
    elif dist == 'braycurtis':
        return distance.braycurtis(vector1, vector2)
    elif dist == 'correlation':
        return distance.correlation(vector1, vector2)
    elif dist == 'canberra':
        return distance.canberra(vector1, vector2)
    elif dist == 'chebyshev':
        return distance.chebyshev(vector1, vector2)
    elif dist == 'cityblock':
        return distance.cityblock(vector1, vector2)
    elif dist == 'minkowski':
        return distance.minkowski(vector1, vector2)
    elif dist == 'sqeuclidean':
        return distance.sqeuclidean(vector1, vector2)
    elif dist == 'cosine':
        return distance.cosine(vector1, vector2)

In [ ]:
df = df[selected_cols]
df.head()

,Generator Output (MW),Lube Oil Outlet Cooler Temp Ct (Deg),No 1 Bearing Oil Drain Temp (Deg),No 1 Turb Bearing Metal Temp A ( Deg ),No 1 Turb Bearing Metal Temp B ( Deg )
0,250.161911,40.001122,48.553509,90.566895,62.404171
1,256.526062,39.968334,48.509892,90.681160,62.096100
2,252.822952,39.993980,48.545334,90.477066,62.382019
3,255.816757,39.974266,48.539497,90.650681,62.200996
4,246.595398,39.990200,48.549404,90.468384,62.456760


In [ ]:
# data shape
df.shape

(52560, 5)

In [ ]:
# define current actual data
current_actual = df.iloc[399,:].values

current_actual

array([393.27600098,  39.96697235,  48.60442352,  90.25898743,
        64.29458618])

In [ ]:
# create state matrix
state_matrix = df.iloc[:400,:].values.T
state_matrix.shape

(5, 400)

In [ ]:
# calculate similarity
sim_vec = []

for i in range(state_matrix.shape[1]):
    sim = 1 - scipy_distance(current_actual, state_matrix[:, i], dist='canberra')
    sim_vec.append(sim)

In [ ]:
# select best 10
top10 = np.sort(np.array(sim_vec).argsort()[::-1][:10])
sim_vec10 = np.array(sim_vec)[top10]

sim_vec10

array([0.99700086, 0.99776908, 0.99687907, 0.9969182 , 0.99678409,
       0.99792758, 0.99749234, 0.99776484, 0.99673775, 1.        ])

In [ ]:
# create dynamic matrix
dynamic_matrix = state_matrix[:, top10]
dynamic_matrix

array([[393.48770142, 394.209198  , 394.59939575, 393.64001465,
        394.11276245, 393.10601807, 392.98602295, 393.16253662,
        392.50515747, 393.27600098],
       [ 40.02999115,  39.99246216,  39.97238159,  39.97789001,
         39.9695015 ,  39.98636627,  39.98130035,  40.01943588,
         40.02780151,  39.96697235],
       [ 48.69527435,  48.65802383,  48.66394043,  48.71835709,
         48.68651581,  48.64979172,  48.67524719,  48.71405792,
         48.66070938,  48.60442352],
       [ 90.26540375,  90.23300171,  90.13049316,  90.18348694,
         90.24081421,  90.2822113 ,  90.2742157 ,  90.23690796,
         90.28652954,  90.25898743],
       [ 64.16958618,  64.29046631,  64.28823853,  64.17978668,
         64.1434021 ,  64.16374207,  64.14723206,  64.2706604 ,
         64.19313049,  64.29458618]])

In [ ]:
# calculate weight
weight = np.array([s/np.sum(sim_vec10) for s in sim_vec10])
weight

array([0.09994722, 0.10002423, 0.09993501, 0.09993893, 0.09992549,
       0.10004012, 0.09999649, 0.1000238 , 0.09992084, 0.10024787])

In [ ]:
# estimate value
estimate_value = np.dot(dynamic_matrix, weight.T)

estimate_value

array([393.5083742 ,  39.99240352,  48.67261341,  90.23921669,
        64.21411102])

In [ ]:
# actual
actual = df.iloc[400,:].values

actual

array([399.72497559,  39.98599625,  48.68819046,  90.43733215,
        64.04458618])

In [ ]:
print(f"MSE: {mean_squared_error(actual, estimate_value)}")
print(f"RMSE: {np.sqrt(mean_squared_error(actual, estimate_value))}")

MSE: 7.742880979886797
RMSE: 2.782603273894214


In [ ]:
# plot y test and y pred
fig = go.Figure()

fig.add_trace(go.Scatter(x=[i for i in range(10)], y=actual,
                         line=dict(color='royalblue'),
                         name='actual value'))
fig.add_trace(go.Scatter(x=[i for i in range(10)], y=estimate_value,
                         line=dict(color='orange'),
                         name='estimate value'))
fig.show()

## Diagnostics

Expressions:
```
if
    (ModelIndications(MECHANICAL,[SVH,SH,VH],[BRG_1_OIL_DRAIN_TEMP])>=1.0
    and
    ModelIndications(MECHANICAL,[SVH,SH,VH],[BRG_1_MTL_TEMP_1, BRG_1_MTL_TEMP_2, BRG_1_MTL_TEMP_3, BRG_1_MTL_TEMP_4])>=1.0)
then 
    Priority(2)
else if
    (ModelIndications(MECHANICAL,[SVH,SH,VH],[BRG_1_OIL_DRAIN_TEMP])>=1.0
    or 
    (ModelIndications(MECHANICAL,[VAR,H],[BRG_1_OIL_DRAIN_TEMP])>=1.0
    and
    ModelIndications(MECHANICAL,[VAR,H],[BRG_1_MTL_TEMP_1, BRG_1_MTL_TEMP_2, BRG_1_MTL_TEMP_3, BRG_1_MTL_TEMP_4])>=1.0))
then 
    Priority(3)
else if
    (ModelIndications(MECHANICAL,[VAR,H],[BRG_1_OIL_DRAIN_TEMP])>=1.0)
then 
    Priority(4)
else
    IsAssociated(MECHANICAL,[GROSS_LOAD,AMBIENT_TEMP,LUBE_OIL_COOLER_OIL_OUTLET_TEMP, LUBE_OIL_COOLER_OIL_OUTLET_PRESS])
```

Diagnostic Functions

https://www.ge.com/digital/documentation/predix-apm/latest/r_apm_smartsignal_exp_syn_diagnostics_functions.html

In [ ]:
def ModelIndications(model, model_tag_rules, asset_tags):
    """Returns a count of the number of model tag advisory indications on a set of model tags

    Parameters
    ----------
    model : str
        A name of a model in the asset blueprint
    model_tag_rules : list
        A list of model blueprint tag rules (mnemonics)
    asset_tags : list
         A list of asset blueprint tags aliases
    """
    pass

In [ ]:
help(ModelIndications)

Help on function ModelIndications in module __main__:

ModelIndications(model, model_tag_rules, asset_tags)
    Returns a count of the number of model tag advisory indications on a set of model tags
    
    Parameters
    ----------
    model : str
        A name of a model in the asset blueprint
    model_tag_rules : list
        A list of model blueprint tag rules (mnemonics)
    asset_tags : list
         A list of asset blueprint tags aliases

